In [1]:
from fastai.text.transform import default_pre_rules, default_post_rules, default_spec_tok

In [2]:
import sentencepiece as spm

In [4]:
import pandas as pd
import numpy as np
import fire
from fastai import F, to_device
import torch
from tqdm import tqdm
import re
import csv
from shutil import copyfile

from functools import reduce
from fastai.text.transform import Tokenizer, BaseTokenizer, Vocab
from fastai.torch_core import *

import shutil
import pathlib
import tarfile
from sklearn import model_selection
from sacremoses import MosesTokenizer
from typing import Dict, Tuple, List

In [10]:
from fastai_contrib.utils import SentencepieceTokenizer, get_sentencepiece

In [5]:
EOS = '<eos>'
UNK = '<unk>'
PAD = '<pad>'
SEP = '<sep>'  # special separator token for NLI
PAD_TOKEN_ID = 1
IMDB, XNLI, TRN, VAL, TST, EN = 'imdb', 'xnli', 'train', 'val', 'test', 'en'
DATASETS = ['imdb', 'xnli']
XNLI_PATHS = {
    TRN: 'XNLI-MT-1.0/multinli/multinli.train.%s.tsv',
    VAL: 'XNLI-1.0/xnli.dev.tsv',
    TST: 'XNLI-1.0/xnli.test.tsv'
}

CLASSES = ['neg', 'pos', 'unsup']

number_match_re = re.compile(r'^([0-9]+[,.]?)+$')
number_split_re = re.compile(r'([,.])')

In [7]:
','.join(default_spec_tok)

'xxbos,xxfld,xxunk,xxpad'

In [9]:
f"--user_defined_symbols={','.join(default_spec_tok)}"

'--user_defined_symbols=xxbos,xxfld,xxunk,xxpad'

In [11]:
tv = get_sentencepiece(path='check', trn_path='data/wiki/ja-2/train.csv', name='wt-2', 
                      vocab_size=8000, model_type='unigram', input_sentence_size=1E7, 
                      pad_idx=PAD_TOKEN_ID, character_coverage=0.995)

In [13]:
v=tv['vocab']

In [14]:
v.itos

['<unk>',
 '<pad>',
 'xxbos',
 'xxfld',
 'xxunk',
 'xxpad',
 '▁"',
 '、',
 '▁0.0,"',
 '。',
 'の',
 '▁',
 'は',
 'を',
 'が',
 'に',
 '・',
 '年',
 ')',
 'で',
 '(',
 'と',
 '.',
 '月',
 'した',
 '」',
 '「',
 'し',
 '1',
 '2',
 'から',
 'である',
 '日',
 'する',
 '3',
 'も',
 'や',
 'ス',
 'として',
 'た',
 '4',
 'る',
 'された',
 '5',
 'では',
 'ア',
 '人',
 '『',
 '』',
 '大',
 'この',
 'な',
 'い',
 '8',
 '6',
 'している',
 'リ',
 '9',
 '年に',
 '7',
 'には',
 'ク',
 '10',
 'り',
 '第',
 'ト',
 'して',
 '-',
 'ル',
 '中',
 'ド',
 'また',
 'て',
 'か',
 'カ',
 'ラ',
 '""',
 'その',
 '山',
 '本',
 'という',
 'され',
 '12',
 'く',
 'ロ',
 'ム',
 '町',
 'ている',
 '日本',
 'など',
 'ら',
 '11',
 'マ',
 'ナ',
 'コ',
 '市',
 '田',
 '上',
 '者',
 '戦',
 '回',
 '長',
 '地',
 's',
 'e',
 'イ',
 '日に',
 '国',
 'フ',
 'ズ',
 'った',
 '一',
 '子',
 'されている',
 'レ',
 'き',
 'S',
 'エ',
 'となった',
 ':',
 '部',
 'A',
 '同',
 'によって',
 '▁-',
 'ン',
 'シ',
 'タ',
 ',',
 '川',
 '下',
 'される',
 'オ',
 'による',
 '後',
 'サ',
 '生',
 '家',
 '小',
 'み',
 '会',
 'リー',
 '18',
 'であった',
 'C',
 '分',
 '出',
 'ウ',
 '名',
 'D',
 '位',
 'B',
 'キ',


In [15]:
t=tv['tokenizer']

- このあとvocab=8000でwt-allで再度sentencepiece trainingを実施
- trainingに使用したsentence数：10,205,438